VACATION

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
cities_df = pd.read_csv('output_data/cities.csv')
cities_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,City country,City datetime
0,0,severo-kurilsk,50.68,156.12,52.21,87,95,14.83,RU,1595970734
1,1,warrington,53.39,-2.58,60.80,72,20,11.41,GB,1595970469
2,2,lata,40.16,-8.33,64.99,94,0,2.30,PT,1595970734
3,3,sao gabriel da cachoeira,-0.13,-67.09,81.59,92,100,1.14,BR,1595970734
4,4,srandakan,-7.94,110.25,73.87,83,69,6.69,ID,1595970734
...,...,...,...,...,...,...,...,...,...,...
560,560,kirillov,59.86,38.38,68.00,82,40,4.47,RU,1595970815
561,561,mitu,1.20,-70.17,78.55,94,100,1.95,CO,1595970815
562,562,ukiah,39.15,-123.21,97.00,21,1,5.82,US,1595970815
563,563,howard springs,-12.50,131.05,64.99,27,27,6.93,AU,1595970815


In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [28]:
locations = cities_df[['Latitude', 'Longitude']].astype(float)
# locations = locations.iloc[0:1]

hum = cities_df['Humidity'].astype(float)
# hum = hum.iloc[0:1]


In [29]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# heat_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
select_cities_df = cities_df.query('70 < Temperature & Temperature < 80 & Cloudiness == 0 & `Wind Speed` < 10' )
select_cities_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,City country,City datetime
5,5,kampene,-3.60,26.67,70.86,36,0,0.40,CD,1595970735
93,93,wuda,39.50,106.71,78.42,26,0,8.19,CN,1595970748
96,96,jadu,31.95,12.03,71.10,50,0,7.23,LY,1595970748
211,211,arraial do cabo,-22.97,-42.03,78.80,78,0,8.05,BR,1595970765
282,282,babstovo,48.12,132.48,70.43,92,0,2.51,RU,1595970776
301,301,janoshalma,46.30,19.33,73.00,57,0,7.74,HU,1595970779
327,327,hazorasp,41.32,61.07,71.60,46,0,6.93,UZ,1595970782
370,370,saquarema,-22.92,-42.51,78.80,78,0,8.05,BR,1595970788
489,489,balkhash,46.84,74.98,70.30,51,0,2.64,KZ,1595970805
496,496,khash,28.22,61.22,74.30,19,0,6.51,IR,1595970806


In [62]:
hotel_df = select_cities_df.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,City country,City datetime,Hotel Name
5,5,kampene,-3.60,26.67,70.86,36,0,0.40,CD,1595970735,
93,93,wuda,39.50,106.71,78.42,26,0,8.19,CN,1595970748,
96,96,jadu,31.95,12.03,71.10,50,0,7.23,LY,1595970748,
211,211,arraial do cabo,-22.97,-42.03,78.80,78,0,8.05,BR,1595970765,
282,282,babstovo,48.12,132.48,70.43,92,0,2.51,RU,1595970776,
301,301,janoshalma,46.30,19.33,73.00,57,0,7.74,HU,1595970779,
327,327,hazorasp,41.32,61.07,71.60,46,0,6.93,UZ,1595970782,
370,370,saquarema,-22.92,-42.51,78.80,78,0,8.05,BR,1595970788,
489,489,balkhash,46.84,74.98,70.30,51,0,2.64,KZ,1595970805,
496,496,khash,28.22,61.22,74.30,19,0,6.51,IR,1595970806,


In [64]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    print(f'city:{row["City"]}')
    lat = row['Latitude']
    lon = row['Longitude']
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
    except (KeyError, IndexError):
        print('Missing field/result... skipping')
        hotel_df.drop(index, inplace=True)

city:kampene
Missing field/result... skipping
city:wuda
city:jadu
city:arraial do cabo
city:babstovo
city:janoshalma
city:hazorasp
city:saquarema
city:balkhash
city:khash


In [65]:
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,City country,City datetime,Hotel Name
93,93,wuda,39.50,106.71,78.42,26,0,8.19,CN,1595970748,Grand Skylight International Hotel Wuhai
96,96,jadu,31.95,12.03,71.10,50,0,7.23,LY,1595970748,فندق السلام
211,211,arraial do cabo,-22.97,-42.03,78.80,78,0,8.05,BR,1595970765,Pousada Oceano de Búzios
282,282,babstovo,48.12,132.48,70.43,92,0,2.51,RU,1595970776,Songjiang International
301,301,janoshalma,46.30,19.33,73.00,57,0,7.74,HU,1595970779,Hotel Galleria
327,327,hazorasp,41.32,61.07,71.60,46,0,6.93,UZ,1595970782,Hotel Uzbekistan
370,370,saquarema,-22.92,-42.51,78.80,78,0,8.05,BR,1595970788,Pousada Laguna
489,489,balkhash,46.84,74.98,70.30,51,0,2.64,KZ,1595970805,Guest House Comfort-House-Balkhash
496,496,khash,28.22,61.22,74.30,19,0,6.51,IR,1595970806,خوابگاه دانشجویی خاش


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [73]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig